In [ ]:
# Cell 1 - Markdown
# Semantic Text Analyzer Demo Notebook

# Cell 2 - Markdown
## Setup and Environment

# Cell 3 - Code
import os
import sys
from pathlib import Path
import asyncio
import logging
from typing import Dict, Any, Optional
import pandas as pd

# Add project root to Python path
project_root = str(Path().resolve().parent)
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to Python path")

# Core components
from src.semantic_analyzer.analyzer import SemanticAnalyzer
from src.utils.FileUtils.file_utils import FileUtils
from src.loaders.parameter_adapter import ParameterAdapter
from src.core.config import AnalyzerConfig

# Initialize FileUtils and set up logging
file_utils = FileUtils()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Cell 4 - Markdown
## Environment Verification

# Cell 5 - Code
def verify_environment():
    """Verify that the notebook environment is properly configured."""
    # Load environment variables
    from dotenv import load_dotenv
    env_path = Path(project_root) / ".env"
    env_loaded = load_dotenv(env_path)

    # Required environment variables
    required_env_vars = [
        'OPENAI_API_KEY',
        'ANTHROPIC_API_KEY',
    ]

    # Basic checks
    basic_checks = {
        "Project root in path": project_root in sys.path,
        "Can import src": "src" in sys.modules,
        "FileUtils initialized": hasattr(file_utils, "project_root"),
        ".env file loaded": env_loaded,
    }

    # Environment variable checks
    env_var_checks = {
        f"{var} set": os.getenv(var) is not None
        for var in required_env_vars
    }

    # Check for required paths using FileUtils
    expected_paths = {
        "Raw data": file_utils.get_data_path("raw"),
        "Processed data": file_utils.get_data_path("processed"),
        "Configuration": file_utils.get_data_path("configurations"),
        "Main config.yaml": Path(project_root) / "config.yaml"
    }
    
    path_checks = {
        f"{name} exists": path.exists()
        for name, path in expected_paths.items()
    }

    # Combine all checks
    all_checks = {
        **basic_checks,
        **env_var_checks,
        **path_checks
    }
    
    # Print results
    print("Environment Check Results:")
    print("=" * 50)
    
    def print_section(title, checks):
        print(f"\n{title}:")
        print("-" * len(title))
        for check, result in checks.items():
            status = "✓" if result else "✗"
            print(f"{status} {check}")
    
    print_section("Basic Setup", basic_checks)
    print_section("Environment Variables", env_var_checks)
    print_section("Project Structure", path_checks)
    
    # Overall status
    all_passed = all(all_checks.values())
    print("\n" + "=" * 50)
    print("Environment Status:", "Ready ✓" if all_passed else "Setup needed ✗")
    
    if not all_passed:
        print("\nSetup Instructions:")
        if not env_loaded:
            print("- Create a .env file in the project root with required API keys")
        for var in required_env_vars:
            if not os.getenv(var):
                print(f"- Add {var} to your .env file")
        for name, path in expected_paths.items():
            if not path.exists():
                print(f"- Create {name} directory at {path}")

    return all_passed

# Run verification
verify_environment()

# Cell 6 - Markdown
## Helper Functions and Test Data Setup

# Cell 7 - Code
class NotebookAnalyzer:
    """Helper class for running analyses in the notebook."""
    
    def __init__(self):
        self.file_utils = FileUtils()
        self.test_texts = self._load_test_texts()
        self.param_adapter = ParameterAdapter(Path(project_root) / "config.yaml")
    
    def _load_test_texts(self) -> Dict[str, str]:
        """Load or create test texts."""
        texts = {
            "technical": """
            Python is a high-level programming language known for its simplicity.
            It supports multiple programming paradigms including procedural and
            object-oriented programming.
            """,
            "business": """
            The company's Q3 results exceeded expectations with revenue growth of 15%.
            Customer acquisition costs decreased while retention rates improved.
            The board has approved a new strategic initiative focusing on expansion.
            """,
            "finnish": """
            Ohjelmistokehittäjä työskentelee asiakasprojektissa kehittäen
            verkkokauppajärjestelmää. Tekninen toteutus vaatii erityistä huomiota
            tietoturvan osalta.
            """
        }
        
        # Save texts using FileUtils for future use
        df = pd.DataFrame([
            {"name": name, "content": content.strip()}
            for name, content in texts.items()
        ])
        
        saved_files, _ = self.file_utils.save_data_to_disk(
            data={"texts": df},
            output_type="raw",
            file_name="test_texts",
            output_filetype="xlsx",
            include_timestamp=False
        )
        
        return texts
    
    async def analyze_text(self, text_key: str, **kwargs):
        """Analyze a specific test text."""
        if text_key not in self.test_texts:
            raise ValueError(f"Unknown text key: {text_key}. Available keys: {list(self.test_texts.keys())}")
        
        analyzer = SemanticAnalyzer(**kwargs)
        results = await analyzer.analyze(self.test_texts[text_key], **kwargs)
        await self.display_results(results)
        return results
    
    async def analyze_all(self, **kwargs):
        """Analyze all test texts."""
        analyzer = SemanticAnalyzer(**kwargs)
        all_results = await analyzer.analyze_batch(
            list(self.test_texts.values()),
            **kwargs
        )
        
        for i, results in enumerate(all_results):
            print(f"\n=== Text {i+1} ===")
            await self.display_results(results)
        
        return all_results
    
    @staticmethod
    async def display_results(results: Dict[str, Any]) -> None:
        """Display analysis results in a formatted way."""
        for analysis_type, data in results.items():
            print(f"\n=== {analysis_type.upper()} ===")
            if isinstance(data, dict):
                for key, value in data.items():
                    print(f"{key}: {value}")
            else:
                print(data)
    
    def save_results(self, results: Dict[str, Any], filename: str) -> Path:
        """Save analysis results using FileUtils."""
        saved_files, _ = self.file_utils.save_data_to_disk(
            data=results,
            output_type="processed",
            file_name=filename,
            output_filetype="yaml"
        )
        return Path(list(saved_files.values())[0])

# Initialize analyzer
notebook_analyzer = NotebookAnalyzer()

# Cell 8 - Markdown
## Basic Analysis Example

# Cell 9 - Code
# Analyze technical text
results = await notebook_analyzer.analyze_text(
    "technical",
    analysis_types=["keywords", "themes"],
    language="en"
)

# Cell 10 - Markdown
## Analysis with Parameters

# Cell 11 - Code
# Load parameters and analyze business text
params = notebook_analyzer.param_adapter.load_and_convert()

results = await notebook_analyzer.analyze_text(
    "business",
    analysis_types=["keywords", "themes", "categories"],
    language="en",
    parameter_file=params
)

# Cell 12 - Markdown
## Finnish Text Analysis

# Cell 13 - Code
# Analyze Finnish text
results = await notebook_analyzer.analyze_text(
    "finnish",
    analysis_types=["keywords", "categories"],
    language="fi"
)

# Cell 14 - Markdown
## Batch Analysis

# Cell 15 - Code
# Analyze all texts
all_results = await notebook_analyzer.analyze_all(
    analysis_types=["keywords", "themes"]
)

# Cell 16 - Markdown
## Saving Results

# Cell 17 - Code
# Save results
saved_path = notebook_analyzer.save_results(results, "analysis_results")
print(f"Results saved to: {saved_path}")

# Cell 18 - Markdown
## Custom Categories Analysis

# Cell 19 - Code
# Define custom categories
categories = {
    "technical": {
        "description": "Technical content",
        "keywords": ["programming", "software", "technology"],
        "threshold": 0.7
    },
    "business": {
        "description": "Business content",
        "keywords": ["revenue", "growth", "financial"],
        "threshold": 0.6
    }
}

# Analyze with custom categories
results = await notebook_analyzer.analyze_text(
    "technical",
    analysis_types=["categories"],
    categories=categories
)